# QCoDeS Example with the Rigol DS 1074 Z oscilloscope

This notebook presents the implementation of QCoDeS driver for the Rigol DS1074Z oscilloscope.

## Importing dependencies 

In [1]:
#Python import 


#Third party imports
import visa

#Qcodes import
from qcodes.instrument_drivers.rigol import DS1074Z
from qcodes.utils.validators import Arrays
from qcodes import initialise_database, load_or_create_experiment, Measurement
from qcodes.dataset.experiment_container import new_experiment

## Create the instrument 

The address of the oscilloscope is needed to create the instrument. The address can be obtained in the following way if the 'physical' instrument is connected. 

In [2]:
rm = visa.ResourceManager()
rigol_address = rm.list_resources()[0]
print(rigol_address)


USB0::0x1AB1::0x04CE::DS1ZB161650342::INSTR


Else the address can also be found using the NI-MAX explorer. The instrument is created in the following way:

In [4]:
rigol = DS1074Z.RigolDrivers('rigol', rigol_address)

NameError: name 'Arrays' is not defined

In [ ]:
rigol.connect_message()

## Prepare trace

In this example we choose channel 1 (ch1) to obtain the trace. We first prepare the oscilloscope to return the values of raw trace.

In [ ]:
rigol.channels.ch1.get_trace.prepare_curvedata()

In [ ]:
trace = rigol.channels.ch1.get_trace

## Trigger setup  

The internal trigger mode type supported by this oscilloscope are `EDGe,PULSe, SLOPe, VIDeo, PATTern, DURATion`. Both the trigger mode and trigger level can be set in the following manner. 

In [ ]:
rigol.trigger_mode('EDGe')
rigol.trigger_level(0.1)

## Data acquisition 

In [ ]:
initialise_database()
exp = load_or_create_experiment(experiment_name='Oscilloscope trace',
                               sample_name='no_name')

In [ ]:
meas = Measurement(exp=exp)

In [ ]:
meas.register_parameter(trace)

In [ ]:
data = rigol.channels.ch1.get_trace.get()




In [ ]:
print(data)

In [ ]:
import qcodes.instrument.sims as sims
# path to the .yaml file containing the simulated instrument
visalib = sims.__file__.replace('__init__.py', 'Rigol1074Z.yaml@sim')

rigol_sim = RigolDrivers('rigol_sim',
                          address='TCPIP0::0.0.0.0::inst0::INSTR',  # This matches the address in the .yaml file
                          visalib=visalib
                          )